# Brone layer

## Order Details

In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE  TABLE order_now
COMMENT 'THE RAW DATA INGESTED'
AS
SELECT * FROM cloud_files(
  "${dataset.path}/order-raw",'parquet',map("schema","order_id STING ,order_timestamp LONG, customer_id STRING, quantity LONG")
)

Customers_data

In [0]:
%sql
CREATE OR REFRESH LIVE TABLE Customers_data
COMMENT 'CUSTOMERS DATA '
AS
SELECT * FROM JSON.`${dataset.path}/customers-json`


## silver layer

In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE TABLE  orders_clean(
  CONSTRAINT valid_order_id EXPECT (order_id is null) on violation drop row)
  comment 'cleaning the customers and orders data'
  as
  select 
  order_id,
  quantity,
  o.customer_id, 
  c.profile:first_name  f_name,
  c.profile:last_name as l_name,
  cast(from_unixtime(order_timestamp,'yyyy-MM-dd HH:mm:ss') as timestamp) as order_timestamp,
  c.profile:address:country as country
  from stream(live.order_now ) o
  left join live.Customers_data c
  on o.customer_id = o.customer_id
  

## Gold layer

In [0]:
%sql
CREATE OR REFRESH LIVE TABLE gold_layer
COMMENT 'GOLD LAYER'
AS
SELECT 
customer_id,
f_name,
l_name,
date_trun("DD",order_timestamp) order_date,
sum(quantity) as books_count
FROM LIVE.orders_clean
WHERE country='china'
GROUP BY customer_id,f_name,l_name,order_date